# RNN from Scratch


In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import d2lzh_pytorch as d2dl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import random

In [15]:
# Obtain Data
def load_data_jay_lyrics():
    with open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_dict, char_list, vocab_size) = load_data_jay_lyrics()

# Load Data
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    num_examples = len(corpus_indices) // num_steps
    num_batch = num_examples // batch_size
    example_list = list(range(num_examples))
    random.shuffle(example_list)
    
    # obtain data in one batch
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(num_batch):
        i = i * batch_size
        batch_indices = example_list[i:i+batch_size]
        X = [_data(j*num_steps) for j in batch_indices]
        Y = [_data(j*num_steps+1) for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device), torch.tensor(Y,dtype=torch.float32,device=device)

def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    batch_num = len(corpus_indices) // batch_size
    # make a matrix over (batch_size, batch_num)
    indices = corpus_indices[0:batch_num*batch_size].view(batch_size,batch_num)
    # dived batch_num by num_steps to get consecutive examples
    batch_num_example = batch_num // num_steps  
    for i in range(batch_num_example):
        i = i * num_steps
        X = corpus_indices[i:i+num_steps]
        Y = corpus_indices[i+1:i+num_steps+1]
        yield X,Y
    
# Define Model
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def init_rnn_state(batch_size, num_hidden, device):
    state = torch.zeros(batch_size, num_hidden, device=device)
    # store the inital state into a tuple
    return (state,)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),dtype=torch.float32,device=device)
        return nn.Parameter(ts,requires_grad=True)
    
    W_ih = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    W_ho = _one((num_hiddens,num_outputs))
    b_h = nn.Parameter(torch.zeros(num_hiddens,device=device),requires_grad=True)
    b_o = nn.Parameter(torch.zeros(num_outputs,device=device),requires_grad=True)
    return nn.ParameterList([W_ih,W_hh,W_ho,b_h,b_o])

def rnn(inputs, state, params):
    W_ih,W_hh,W_ho,b_h,b_o = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X,W_ih) + torch.matmul(H,W_hh) + b_h)
        Y = torch.matmul(H,W_ho) + b_o
        outputs.append(Y)
    return outputs, (H,)

def grad_clipping(params,clipping_threshold,device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > clipping_threshold:
        for param in params:
            param.grad.data *= (clipping_threshold / norm)


In [19]:
# Train Model
def predict(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size,\
                device, char_list, char_dict):
    state = init_rnn_state(1,num_hiddens,device)
    #print(prefix[0])
    output = [char_dict[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X = d2dl.to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state) = rnn(X,state,params)
        if t < len(prefix) - 1:
            output.append(char_dict[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(1).item()))
    return ''.join([char_list[i] for i in output])
    
    
def train_and_predict(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device,\
                     corpus_indices, char_list, char_dict, is_random, num_epochs,\
                     num_steps, lr, clipping_theta, batch_size, pred_period, pred_len,\
                     prefixes, data_iter_random, data_iter_consecutive, grad_clipping):
    if is_random:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    
    params = get_params()
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random:
            state = init_rnn_state(batch_size,num_hiddens,device)
        l_sum, n = 0.0,0
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device=None)
        for X,Y in data_iter:
            if is_random:
                state = init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
            # computation graph
            inputs = d2dl.to_onehot(X,vocab_size)
            (outputs,state) = rnn(inputs,state,params)
            outputs = torch.cat(outputs,dim=0)
            y = torch.transpose(Y,0,1).contiguous().view(-1)
            l = loss(outputs,y.long())
        
            # set gradient to zero manually
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
        
            l.backward()
            grad_clipping(params,clipping_theta,device)
            d2dl.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
    
        if (epoch+1) % pred_period == 0:
            print('epoch: %d, perplexity: %f' %(epoch+1,math.exp(l_sum/n)))
            for prefix in prefixes:
                print('-',predict(prefix, pred_len, rnn, params, init_rnn_state,\
                              num_hiddens, vocab_size, device, char_list, char_dict))
        
        
        
            
        

In [20]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, char_list,
                      char_dict, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes,data_iter_random,data_iter_consecutive,grad_clipping)

epoch: 50, perplexity: 73.483295
分
- 分开 我不要 一九两 我给就 一九四 我什么 一九四 我什么 一颗四颗三步四 我什么 一颗两颗三颗四 我
不
- 不分开 我有么 一颗两颗三步四 我什么 一颗两颗三颗四 我什么 一颗两颗三颗四 我什么 一颗两颗三颗四 我
epoch: 100, perplexity: 10.561730
分
- 分开 有一个人 在使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼
不
- 不分开吗 我不想我想你 你你想觉 我已好这生活 后知后觉 我该好这生活 后知后觉 我该好这生活 后知后觉 
epoch: 150, perplexity: 2.905499
分
- 分开 有亮底 一颗两步三步四颗 连成线背著背默默许下心愿 看远方的星是否听的见 还牵灌 一步两步三步四步
不
- 不分开扫 我后你的 你在后真 你却没用 不够不痛 你不懂 连不知的你 一阵莫名 恨你在美太 你一定烛抽 仙
epoch: 200, perplexity: 1.617301
分
- 分开 沙亮开 告诉我 印地 是术完闷  所底梦  你爱一起热粥 没伤了  如果我想 你不作爸不嘛 我知后
不
- 不分开期 我叫你爸 你打我妈 这样对吗了嘛这样 我不下神之你许愿知道好  我的世界已狂风暴雨 Wu 看不你
epoch: 250, perplexity: 1.323383
分
- 分开 沙杰开不 我有了带节 巫的事空 我想要 我情走 你怎么 但分怎 三沉丹田 心头之中 最上心动 染红
不
- 不分开期把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 一根我不 我被店爸恼  没有你烦


## RNN using PyTorch

In [ ]:
# Obtain Data
(corpus_indices, char_dict, char_list, vocab_size) = load_data_jay_lyrics()

# Define Model
rnn_layer = nn.RNN(vocab_size,hidden_size)
class rnnModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(rnnModel,self).__init__()
        self.rnn = rnn_layer()
        